[자연어처리] 텍스트 분류 모델 구현

- 데이터 셋 : 200개 한국어 뉴스 기사
- 라 벨 : 정치(0), 경제(1), 사회(2), 생활/문화(3), 세계(4), 기술/IT(5), 연예(6), 스포츠(7)

- 입력 받은 TEXT에 대한 기사 종류 분류 결과 출력까지 완성

[1] 모듈 로딩 및 데이터 준비 <hr>

In [1]:
## 모듈 로딩
import sys
sys.path.append(r'C:\Users\KDP-50\OneDrive\바탕 화면\KDT_DYH\10.Natural_Language_Processing')

from Korpora import Korpora                         # Open Korean Dataset
from konlpy.tag import *                            # 형태소 분석기
import spacy                                        # 형태소 분석기

from nltk.tokenize import word_tokenize, sent_tokenize, wordpunct_tokenize
from torch.utils.data import Dataset, DataLoader    # Pytorch Dataset 관련 모듈
from nltk.corpus import stopwords
import pandas as pd
import numpy as np
import re
from konlpy.tag import *

# Pytorch 관련 모듈
from torch import nn
from torch import optim
import torch
from torch.utils.data import TensorDataset, DataLoader

from NLPmodules import classRNN
from NLPmodules import funcRNN

In [2]:
## 데이터 준비
TEXT_PATH = '../NLParticle/0/concatNews0.txt'
with open(TEXT_PATH, mode='r', encoding='utf-8') as f:
    texts = f.read().splitlines()

In [3]:
## 언어 모델 설정
KO_MODEL = 'ko_core_news_sm'

### 한국어 분석기 생성
nlp = spacy.load(KO_MODEL)

### 형태소 분석기
# 한나눔
han = Hannanum()

# 꼬꼬마
kkma = Kkma()

# OKT
okt = Okt()

In [4]:
# 불용어 경로
STOPWORDS = '../Data/basic_ko_stopwords.txt'
with open(STOPWORDS, mode='r', encoding='utf-8') as f:
    stopwords_ko = f.read().splitlines()

In [5]:
texts[:10]

["동남아 담당' 北 최희철 부상 베이징 도착…싱가포르행 주목\t최 부상, 행선지·방문 목적 질문에는 '묵묵부답'",
 '',
 '(베이징=연합뉴스) 김진방 특파원 = 북한이 북미 정상회담 무산 가능성까지 거론하며 강경한 태도를 보이는 가운데 동남아시아 외교를 담당하는 최희철 북한 외무성 부상이 19일 중국 베이징 서우두(首都) 공항에 모습을 드러냈다.',
 '',
 '최 부상은 이날 오전 평양발 고려항공 JS151편을 이용해 베이징 서우두 공항에 도착했다.',
 '',
 '최 부상은 최종 목적지를 묻는 취재진의 질문에 아무런 답변을 하지 않고, 북한 대사관 관계자들과 함께 공항을 빠져나갔다.',
 '',
 '북미 정상회담을 20여 일 앞둔 상황에서 동남아 외교통인 최 부상이 정상회담 준비 등을 위해 회담 개최 예정지인 싱가포르를 방문할 가능성도 제기되고 있다.',
 '']

In [6]:
strip_texts=[]
for text in texts:
    if len(text):
        strip_texts.append(text)

In [7]:
strip_texts[:10]

["동남아 담당' 北 최희철 부상 베이징 도착…싱가포르행 주목\t최 부상, 행선지·방문 목적 질문에는 '묵묵부답'",
 '(베이징=연합뉴스) 김진방 특파원 = 북한이 북미 정상회담 무산 가능성까지 거론하며 강경한 태도를 보이는 가운데 동남아시아 외교를 담당하는 최희철 북한 외무성 부상이 19일 중국 베이징 서우두(首都) 공항에 모습을 드러냈다.',
 '최 부상은 이날 오전 평양발 고려항공 JS151편을 이용해 베이징 서우두 공항에 도착했다.',
 '최 부상은 최종 목적지를 묻는 취재진의 질문에 아무런 답변을 하지 않고, 북한 대사관 관계자들과 함께 공항을 빠져나갔다.',
 '북미 정상회담을 20여 일 앞둔 상황에서 동남아 외교통인 최 부상이 정상회담 준비 등을 위해 회담 개최 예정지인 싱가포르를 방문할 가능성도 제기되고 있다.',
 '최 부상은 지난 3월에도 아세안(ASEAN·동남아시아국가연합) 의장국이기도 한 싱가포르를 방문해 양국관계와 올해 8월 열리는 아세안지역안보포럼(ARF) 의제 등을 논의한 바 있다.',
 '또 지난해 북핵 문제를 두고 북미 간 긴장관계가 형성됐을 때도 ARF에 참석해 아세안을 상대로 여론전을 펼쳤다. 북한의 초청으로 비자이 쿠마르 싱 인도 외교부 국무장관이 방북했을 때도 최 부상은 싱 국무장관을 직접 영접하고, 한반도 문제를 논의하기도 했다.',
 '베이징 소식통은 "최 부상이 대(對)미 외교담당이 아니기 때문에 싱가포르로 갈 가능성이 큰 것은 아니다"며 "만약 싱가포르에 간다면 정상회담과 관련한 지원 작업 준비 등을 위한 것일 가능성이 크다"고 말했다.동남아 담당\' 北 최희철 부상 베이징 도착…싱가포르행 주목\t최 부상, 행선지·방문 목적 질문에는 \'묵묵부답\'(베이징=연합뉴스) 김진방 특파원 = 북한이 북미 정상회담 무산 가능성까지 거론하며 강경한 태도를 보이는 가운데 동남아시아 외교를 담당하는 최희철 북한 외무성 부상이 19일 중국 베이징 서우두(首都) 공항에 모습을 드러냈다.최 부상은 이날 오전 평양발 고려항공 JS151

In [91]:
new_list=[]

for i in range(10):
    new_text="" # 단어 더하는 변수 (for문 돌때마다 리셋)
    for textList in strip_texts[i]:
        for text in textList:
            text = re.sub('[^가-힣]','\n',text) # 순수 한글만 가져오기
            new_text += text # 한 글자들 단어로 만들기
        result_text = new_text.split('\n')
        # 빈 문자열 (공백) 제거
        result_text = [t.strip() for t in result_text if t.strip()]
        # print(result_text)
    if len(result_text): new_list.append(result_text)
for textList in new_list:
    print(textList, end='\n')

['동남아', '담당', '최희철', '부상', '베이징', '도착', '싱가포르행', '주목', '최', '부상', '행선지', '방문', '목적', '질문에는', '묵묵부답']
['베이징', '연합뉴스', '김진방', '특파원', '북한이', '북미', '정상회담', '무산', '가능성까지', '거론하며', '강경한', '태도를', '보이는', '가운데', '동남아시아', '외교를', '담당하는', '최희철', '북한', '외무성', '부상이', '일', '중국', '베이징', '서우두', '공항에', '모습을', '드러냈다']
['최', '부상은', '이날', '오전', '평양발', '고려항공', '편을', '이용해', '베이징', '서우두', '공항에', '도착했다']
['최', '부상은', '최종', '목적지를', '묻는', '취재진의', '질문에', '아무런', '답변을', '하지', '않고', '북한', '대사관', '관계자들과', '함께', '공항을', '빠져나갔다']
['북미', '정상회담을', '여', '일', '앞둔', '상황에서', '동남아', '외교통인', '최', '부상이', '정상회담', '준비', '등을', '위해', '회담', '개최', '예정지인', '싱가포르를', '방문할', '가능성도', '제기되고', '있다']
['최', '부상은', '지난', '월에도', '아세안', '동남아시아국가연합', '의장국이기도', '한', '싱가포르를', '방문해', '양국관계와', '올해', '월', '열리는', '아세안지역안보포럼', '의제', '등을', '논의한', '바', '있다']
['또', '지난해', '북핵', '문제를', '두고', '북미', '간', '긴장관계가', '형성됐을', '때도', '에', '참석해', '아세안을', '상대로', '여론전을', '펼쳤다', '북한의', '초청으로', '비자이', '쿠마르', '싱', '인도', '외교부', '국무장관이', '방북했을', '때도', '최', '부상은', '싱', '국무장관을

In [31]:
def generateRE(texts):
    new_list=[]

    # for i in range(len(texts)):
    for i in range(10):
        new_text="" # 단어 더하는 변수 (for문 돌때마다 리셋)
        for textList in strip_texts[i]:
            for text in textList:
                text = re.sub('[^가-힣]','\n',text) # 순수 한글만 가져오기
                new_text += text # 한 글자들 단어로 만들기
            result_text=new_text.split('\n')
            # 빈 문자열 (공백) 제거
            result_text = [t.strip() for t in result_text if t.strip()]
            # print(result_text)
        if len(result_text): new_list.append(result_text)
    yield new_list

In [36]:
new_list = generateRE(strip_texts)

# label 만들기
text_dict={}

# 이중 리스트 접근 [[0번 파일],[1번 파일],...,]
for textfiles in new_list:
    # [0번 파일] 접근, [1번 파일] 접근, ...
    for textfile in textfiles:
        # 파일 내의 텍스트 라벨링
        for text in textfile:
            if text not in textList:
                textList[text] = '0'

In [37]:
text_dict

{'동남아': '0',
 '담당': '0',
 '최희철': '0',
 '부상': '0',
 '베이징': '0',
 '도착': '0',
 '싱가포르행': '0',
 '주목': '0',
 '최': '0',
 '행선지': '0',
 '방문': '0',
 '목적': '0',
 '질문에는': '0',
 '묵묵부답': '0',
 '연합뉴스': '0',
 '김진방': '0',
 '특파원': '0',
 '북한이': '0',
 '북미': '0',
 '정상회담': '0',
 '무산': '0',
 '가능성까지': '0',
 '거론하며': '0',
 '강경한': '0',
 '태도를': '0',
 '보이는': '0',
 '가운데': '0',
 '동남아시아': '0',
 '외교를': '0',
 '담당하는': '0',
 '북한': '0',
 '외무성': '0',
 '부상이': '0',
 '일': '0',
 '중국': '0',
 '서우두': '0',
 '공항에': '0',
 '모습을': '0',
 '드러냈다': '0',
 '부상은': '0',
 '이날': '0',
 '오전': '0',
 '평양발': '0',
 '고려항공': '0',
 '편을': '0',
 '이용해': '0',
 '도착했다': '0',
 '최종': '0',
 '목적지를': '0',
 '묻는': '0',
 '취재진의': '0',
 '질문에': '0',
 '아무런': '0',
 '답변을': '0',
 '하지': '0',
 '않고': '0',
 '대사관': '0',
 '관계자들과': '0',
 '함께': '0',
 '공항을': '0',
 '빠져나갔다': '0',
 '정상회담을': '0',
 '여': '0',
 '앞둔': '0',
 '상황에서': '0',
 '외교통인': '0',
 '준비': '0',
 '등을': '0',
 '위해': '0',
 '회담': '0',
 '개최': '0',
 '예정지인': '0',
 '싱가포르를': '0',
 '방문할': '0',
 '가능성도': '0',
 '제기되고': '0',
 '있다': '0',


In [38]:
texts=[]
labels=[]
for key, value in text_dict.items():
    if len(key):
        texts.append(key)
        labels.append(value)

In [41]:
texts[:10], labels[:10]

(['동남아', '담당', '최희철', '부상', '베이징', '도착', '싱가포르행', '주목', '최', '행선지'],
 ['0', '0', '0', '0', '0', '0', '0', '0', '0', '0'])

In [42]:
class TextDataset(Dataset):
    def __init__(self, feature, label):
        self.feature = feature
        self.label = label
        self.n_rows = len(feature)

    def __len__(self):
        return self.n_rows

    def __getitem__(self, index):
        return self.feature[index], self.label[index]

In [43]:
textDS = TextDataset(texts, labels)

- 토큰화

In [80]:
def generateToken(dataset):
    for text, label in dataset:
    # for text in dataset:
        token_list = []
        doc = nlp(text)

        for token in doc:
            if (not token.is_punct) and (not token.is_stop) and (not token in stopwords_ko): # 불용어 제거 파트
                token=re.sub('[^가-힣]','',str(token)) # 정규식 사용 한국어 외에 필요없는 토큰 제거
                if len(token): token_list.append(str(token))
        yield token_list

In [81]:
token_generator = generateToken(textDS)

# 토큰을 키로 해서 빈도수 저장
token_freq={} # dict에서는 key값을 찾아봄

# 라인(줄)별 토큰을 읽어서 빈도 체크
for tokenList in token_generator:
    for token in tokenList:
        # 불용어 재차 제거
        if token not in stopwords_ko:
            # 카운트시, 토큰 key가 존재하지 않으면 key 추가
            if token not in token_freq:
                token_freq[token] = 1
            # 카운트시, 이미 존재하는 토큰 key는 1 증가
            else:
                token_freq[token] += 1
print(token_freq)

{'동남아': 1, '담당': 1, '최희철': 1, '부상': 1, '베이징': 1, '도착': 1, '싱가포르행': 1, '주목': 1, '행선지': 1, '방문': 1, '목적': 1, '질문에는': 1, '묵묵부답': 1, '연합뉴스': 1, '김진방': 1, '특파원': 1, '북한이': 1, '북미': 1, '정상회담': 1, '무산': 1, '가능성까지': 1, '거론하며': 1, '강경한': 1, '태도를': 1, '보이는': 1, '가운데': 1, '동남아시아': 1, '외교를': 1, '담당하는': 1, '북한': 1, '외무성': 1, '부상이': 1, '중국': 1, '서우두': 1, '공항에': 1, '모습을': 1, '드러냈다': 1, '부상은': 1, '이날': 1, '오전': 1, '평양발': 1, '고려항공': 1, '편을': 1, '이용해': 1, '도착했다': 1, '최종': 1, '목적지를': 1, '묻는': 1, '취재진의': 1, '질문에': 1, '답변을': 1, '하지': 1, '않고': 1, '대사관': 1, '관계자들과': 1, '함께': 1, '공항을': 1, '빠져나갔다': 1, '정상회담을': 1, '앞둔': 1, '상황에서': 1, '외교통인': 1, '준비': 1, '등을': 1, '회담': 1, '개최': 1, '예정지인': 1, '싱가포르를': 1, '방문할': 1, '가능성도': 1, '제기되고': 1, '있다': 1, '지난': 1, '월에도': 1, '아세안': 1, '동남아시아국가연합': 1, '의장국이기도': 1, '방문해': 1, '양국관계와': 1, '올해': 1, '월': 1, '열리는': 1, '아세안지역안보포럼': 1, '의제': 1, '논의한': 1, '지난해': 1, '북핵': 1, '문제를': 1, '두고': 1, '간': 1, '긴장관계가': 1, '형성됐을': 1, '때도': 1, '참석해': 1, '아세안을': 1, '상대로': 1, '여론전을': 1, '펼쳤다': 1, '

In [82]:
token_generator = generateToken(textDS)
i = 0

for token_list in token_generator:
    i += 1
    for token in token_list:
        print(repr(token))
    if i == 5:
        break

'동남아'
'담당'
'최희철'
'부상'
'베이징'


- 단어별 빈도수 계산

In [83]:
token_generator = generateToken(textDS)

# 토큰을 키로 해서 빈도수 저장
token_freq={} # dict에서는 key값을 찾아봄

# 라인(줄)별 토큰을 읽어서 빈도 체크
for tokenList in token_generator:
    for token in tokenList:
        # 불용어 재차 제거
        if token not in stopwords_ko:
            # 카운트시, 토큰 key가 존재하지 않으면 key 추가
            if token not in token_freq:
                token_freq[token] = 1
            # 카운트시, 이미 존재하는 토큰 key는 1 증가
            else:
                token_freq[token] += 1
print(token_freq)

{'동남아': 1, '담당': 1, '최희철': 1, '부상': 1, '베이징': 1, '도착': 1, '싱가포르행': 1, '주목': 1, '행선지': 1, '방문': 1, '목적': 1, '질문에는': 1, '묵묵부답': 1, '연합뉴스': 1, '김진방': 1, '특파원': 1, '북한이': 1, '북미': 1, '정상회담': 1, '무산': 1, '가능성까지': 1, '거론하며': 1, '강경한': 1, '태도를': 1, '보이는': 1, '가운데': 1, '동남아시아': 1, '외교를': 1, '담당하는': 1, '북한': 1, '외무성': 1, '부상이': 1, '중국': 1, '서우두': 1, '공항에': 1, '모습을': 1, '드러냈다': 1, '부상은': 1, '이날': 1, '오전': 1, '평양발': 1, '고려항공': 1, '편을': 1, '이용해': 1, '도착했다': 1, '최종': 1, '목적지를': 1, '묻는': 1, '취재진의': 1, '질문에': 1, '답변을': 1, '하지': 1, '않고': 1, '대사관': 1, '관계자들과': 1, '함께': 1, '공항을': 1, '빠져나갔다': 1, '정상회담을': 1, '앞둔': 1, '상황에서': 1, '외교통인': 1, '준비': 1, '등을': 1, '회담': 1, '개최': 1, '예정지인': 1, '싱가포르를': 1, '방문할': 1, '가능성도': 1, '제기되고': 1, '있다': 1, '지난': 1, '월에도': 1, '아세안': 1, '동남아시아국가연합': 1, '의장국이기도': 1, '방문해': 1, '양국관계와': 1, '올해': 1, '월': 1, '열리는': 1, '아세안지역안보포럼': 1, '의제': 1, '논의한': 1, '지난해': 1, '북핵': 1, '문제를': 1, '두고': 1, '간': 1, '긴장관계가': 1, '형성됐을': 1, '때도': 1, '참석해': 1, '아세안을': 1, '상대로': 1, '여론전을': 1, '펼쳤다': 1, '

In [84]:
# 단어들 별로 빈도수 저장
freqsDict={}
for k, v in token_freq.items():
    if v not in freqsDict:
        freqsDict[v]=[k]
    else:
        freqsDict[v].append(k)
print(freqsDict)

{1: ['동남아', '담당', '최희철', '부상', '베이징', '도착', '싱가포르행', '주목', '행선지', '방문', '목적', '질문에는', '묵묵부답', '연합뉴스', '김진방', '특파원', '북한이', '북미', '정상회담', '무산', '가능성까지', '거론하며', '강경한', '태도를', '보이는', '가운데', '동남아시아', '외교를', '담당하는', '북한', '외무성', '부상이', '중국', '서우두', '공항에', '모습을', '드러냈다', '부상은', '이날', '오전', '평양발', '고려항공', '편을', '이용해', '도착했다', '최종', '목적지를', '묻는', '취재진의', '질문에', '답변을', '하지', '않고', '대사관', '관계자들과', '함께', '공항을', '빠져나갔다', '정상회담을', '앞둔', '상황에서', '외교통인', '준비', '등을', '회담', '개최', '예정지인', '싱가포르를', '방문할', '가능성도', '제기되고', '있다', '지난', '월에도', '아세안', '동남아시아국가연합', '의장국이기도', '방문해', '양국관계와', '올해', '월', '열리는', '아세안지역안보포럼', '의제', '논의한', '지난해', '북핵', '문제를', '두고', '간', '긴장관계가', '형성됐을', '때도', '참석해', '아세안을', '상대로', '여론전을', '펼쳤다', '북한의', '초청으로', '비자이', '쿠마르', '싱', '인도', '외교부', '국무장관이', '방북했을', '국무장관을', '직접', '영접하고', '한반도', '논의하기도', '했다', '소식통은', '외교담당이', '아니기', '싱가포르로', '갈', '가능성이', '큰', '것은', '아니다', '싱가포르에', '간다면', '정상회담과', '관련한', '지원', '작업', '위한', '것일', '크다', '말했다']}


In [85]:
# 단어등장 횟수, 단어 개수 및 단어이름 별로 빈도수 저장
freqsDict={}
for k, v in token_freq.items():
    if v not in freqsDict:
        # freqsDict[v]=[k]
        # freqsDict[v]=1
        freqsDict[v]=[1, [k]]
    else:
        # freqsDict[v].append(k)
        # freqsDict[v]+=1
        freqsDict[v][0]+=1
        freqsDict[v][1].append(k)
print(freqsDict)

{1: [132, ['동남아', '담당', '최희철', '부상', '베이징', '도착', '싱가포르행', '주목', '행선지', '방문', '목적', '질문에는', '묵묵부답', '연합뉴스', '김진방', '특파원', '북한이', '북미', '정상회담', '무산', '가능성까지', '거론하며', '강경한', '태도를', '보이는', '가운데', '동남아시아', '외교를', '담당하는', '북한', '외무성', '부상이', '중국', '서우두', '공항에', '모습을', '드러냈다', '부상은', '이날', '오전', '평양발', '고려항공', '편을', '이용해', '도착했다', '최종', '목적지를', '묻는', '취재진의', '질문에', '답변을', '하지', '않고', '대사관', '관계자들과', '함께', '공항을', '빠져나갔다', '정상회담을', '앞둔', '상황에서', '외교통인', '준비', '등을', '회담', '개최', '예정지인', '싱가포르를', '방문할', '가능성도', '제기되고', '있다', '지난', '월에도', '아세안', '동남아시아국가연합', '의장국이기도', '방문해', '양국관계와', '올해', '월', '열리는', '아세안지역안보포럼', '의제', '논의한', '지난해', '북핵', '문제를', '두고', '간', '긴장관계가', '형성됐을', '때도', '참석해', '아세안을', '상대로', '여론전을', '펼쳤다', '북한의', '초청으로', '비자이', '쿠마르', '싱', '인도', '외교부', '국무장관이', '방북했을', '국무장관을', '직접', '영접하고', '한반도', '논의하기도', '했다', '소식통은', '외교담당이', '아니기', '싱가포르로', '갈', '가능성이', '큰', '것은', '아니다', '싱가포르에', '간다면', '정상회담과', '관련한', '지원', '작업', '위한', '것일', '크다', '말했다']]}


In [86]:
# [(단어나온 횟수, [단어개수, [단어이름]]) ...]
sorted(freqsDict.items(), reverse=True)

[(1,
  [132,
   ['동남아',
    '담당',
    '최희철',
    '부상',
    '베이징',
    '도착',
    '싱가포르행',
    '주목',
    '행선지',
    '방문',
    '목적',
    '질문에는',
    '묵묵부답',
    '연합뉴스',
    '김진방',
    '특파원',
    '북한이',
    '북미',
    '정상회담',
    '무산',
    '가능성까지',
    '거론하며',
    '강경한',
    '태도를',
    '보이는',
    '가운데',
    '동남아시아',
    '외교를',
    '담당하는',
    '북한',
    '외무성',
    '부상이',
    '중국',
    '서우두',
    '공항에',
    '모습을',
    '드러냈다',
    '부상은',
    '이날',
    '오전',
    '평양발',
    '고려항공',
    '편을',
    '이용해',
    '도착했다',
    '최종',
    '목적지를',
    '묻는',
    '취재진의',
    '질문에',
    '답변을',
    '하지',
    '않고',
    '대사관',
    '관계자들과',
    '함께',
    '공항을',
    '빠져나갔다',
    '정상회담을',
    '앞둔',
    '상황에서',
    '외교통인',
    '준비',
    '등을',
    '회담',
    '개최',
    '예정지인',
    '싱가포르를',
    '방문할',
    '가능성도',
    '제기되고',
    '있다',
    '지난',
    '월에도',
    '아세안',
    '동남아시아국가연합',
    '의장국이기도',
    '방문해',
    '양국관계와',
    '올해',
    '월',
    '열리는',
    '아세안지역안보포럼',
    '의제',
    '논의한',
    '지난해',
    '북핵',
    '문제를',
  

In [87]:
## 빈도가 높게 나오는 순서대로 단어 정렬
#                k, v -----------------------------|
storedTokens=sorted(token_freq.items(), reverse=True, key=lambda x:x[1])

In [88]:
## 단어사전 생성 및 초기화
## 특수토큰 : 'PAD', 'OOV' 또는 'UNK'
PAD_TOKEN, OOV_TOKEN='PAD', 'OOV'
vocab={PAD_TOKEN:0, OOV_TOKEN:1}

In [89]:
## 데이터 코퍼스에서 추출된 토큰(단어)들
# vocab[]

for idx, tk in enumerate(storedTokens, 2):
    vocab[tk[0]] = idx
print(vocab)

{'PAD': 0, 'OOV': 1, '동남아': 2, '담당': 3, '최희철': 4, '부상': 5, '베이징': 6, '도착': 7, '싱가포르행': 8, '주목': 9, '행선지': 10, '방문': 11, '목적': 12, '질문에는': 13, '묵묵부답': 14, '연합뉴스': 15, '김진방': 16, '특파원': 17, '북한이': 18, '북미': 19, '정상회담': 20, '무산': 21, '가능성까지': 22, '거론하며': 23, '강경한': 24, '태도를': 25, '보이는': 26, '가운데': 27, '동남아시아': 28, '외교를': 29, '담당하는': 30, '북한': 31, '외무성': 32, '부상이': 33, '중국': 34, '서우두': 35, '공항에': 36, '모습을': 37, '드러냈다': 38, '부상은': 39, '이날': 40, '오전': 41, '평양발': 42, '고려항공': 43, '편을': 44, '이용해': 45, '도착했다': 46, '최종': 47, '목적지를': 48, '묻는': 49, '취재진의': 50, '질문에': 51, '답변을': 52, '하지': 53, '않고': 54, '대사관': 55, '관계자들과': 56, '함께': 57, '공항을': 58, '빠져나갔다': 59, '정상회담을': 60, '앞둔': 61, '상황에서': 62, '외교통인': 63, '준비': 64, '등을': 65, '회담': 66, '개최': 67, '예정지인': 68, '싱가포르를': 69, '방문할': 70, '가능성도': 71, '제기되고': 72, '있다': 73, '지난': 74, '월에도': 75, '아세안': 76, '동남아시아국가연합': 77, '의장국이기도': 78, '방문해': 79, '양국관계와': 80, '올해': 81, '월': 82, '열리는': 83, '아세안지역안보포럼': 84, '의제': 85, '논의한': 86, '지난해': 87, '북핵': 88, '문제를': 89, '두

In [77]:
token_generator = generateToken(textDS)

## 텍스트 문장 ===> 수치화 [인코딩]
encodingData=[]
for tokenList in token_generator:
    sent=[]
    print(f'문장: {tokenList}')
    for token in tokenList:
        sent.append(vocab[token])

    # 인코딩 된 문장 저장
    encodingData.append(sent)
    print(f'==>인코딩: {sent}\n')

문장: ['동남아']
==>인코딩: [2]

문장: ['담당']
==>인코딩: [3]

문장: ['최희철']
==>인코딩: [4]

문장: ['부상']
==>인코딩: [5]

문장: ['베이징']
==>인코딩: [6]

문장: ['도착']
==>인코딩: [7]

문장: ['싱가포르행']
==>인코딩: [8]

문장: ['주목']
==>인코딩: [9]

문장: ['최']


KeyError: '최'

In [ ]:
# 문장들 마다 길이 체크
# max([len(sent) for sent in encodingData])
dataLen=[len(sent) for sent in encodingData]
print(dataLen)

In [ ]:
import matplotlib.pyplot as plt
plt.hist(dataLen, bins=15)
plt.show()

In [ ]:
## 기준 길이 (가장 긴 길이)
MAX_LENGTH = max(dataLen)
MAX_LENGTH

In [37]:
### [1] 가장 긴 문장 길이 기준
for idx, sent in enumerate(encodingData):
    cur_length=len(sent)
    if cur_length < MAX_LENGTH:
        # sent=sent+([0] * (MAX_LENGTH-cur_length)) # 짧은 만큼 0으로 채워줘야함.
        encodingData[idx]=sent+([0] * (MAX_LENGTH-cur_length)) # enumerate로 index 접근

In [ ]:
for _ in encodingData[:3]: print(len(_), _)

In [61]:
class TextDataset(Dataset):
    def __init__(self, feature, label):
        self.feature = feature
        self.label = label
        self.n_rows = len(feature)

    def __len__(self):
        return self.n_rows

    def __getitem__(self, index):
        # 텐서화
        featureTS=torch.tensor(self.feature[index])
        targetTS=torch.tensor(int(self.label[index]))

        # 피쳐와 타겟 반환
        return featureTS, targetTS

In [62]:
textDS = TextDataset(encodingData, labels)

In [63]:
# token to id / id to token
token_to_id = {token: idx for idx, token in enumerate(vocab)}
id_to_token = {idx: token for idx, token in enumerate(vocab)}

In [64]:
# 조건 지정
device = "cuda" if torch.cuda.is_available() else "cpu"
epochs = 5
interval = 500

In [65]:
# 손실 함수와 최적화 함수 정의
n_vocab = len(token_to_id)
hidden_dim = 64
embedding_dim = 128
n_layers = 2

In [77]:
# 데이터로더 적용
train_loader = DataLoader(textDS, batch_size=8, shuffle=True)

In [ ]:
classifier = classRNN.SentenceClassifier(
    n_vocab=n_vocab, hidden_dim=hidden_dim, embedding_dim=embedding_dim, n_layers=n_layers
)

# 손실 함수 지정
# criterion = nn.BCEWithLogitsLoss()
criterion = nn.CrossEntropyLoss()

# 최적화 함수 지정
optimizer = optim.RMSprop(classifier.parameters(), lr=0.001)

# 학습 진행
for epoch in range(epochs):
    funcRNN.trainModel(classifier, train_loader, criterion, optimizer, device, interval)
    # funcRNN.testModel(classifier, test_loader, criterion, device)